In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import BertTokenizer, BertModel
import numpy as np
import joblib
from sklearn.impute import SimpleImputer

In [ ]:
# dt_res = pd.read_csv('/content/drive/MyDrive/Competitions/test_dataset_vprod_test/TEST_RES.csv')
# dt_sal = pd.read_csv('/content/drive/MyDrive/Competitions/test_dataset_vprod_test/TEST_SAL.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dt_sal = pd.read_csv('/content/drive/MyDrive/ColabFiles/TEST_SAL.csv')

# тестирование берта

In [ ]:
dt_res.head()

In [ ]:
target_column = 'job_title'  # Столбец с названиями должностей
text_columns = ['achievements_modified', 'demands', 'company_name']  # Текстовые столбцы

In [ ]:
dt_res['text'] = dt_res[text_columns].fillna('').apply(lambda x: '. '.join(x), axis=1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# prompt: напиши код чтобы протестировать rubert с hugging face претренированный мной

# Assuming you have a pre-trained RuBERT model and tokenizer
model_name = "/content/drive/MyDrive/Competitions/rubert-model"  # Replace with your model path
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to predict sentiment
def predict_sentiment(text):
  inputs = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    outputs = model(**inputs)
  predicted_class = torch.argmax(outputs.logits, dim=1).item()
  return predicted_class

# Apply sentiment prediction to the 'text' column
dt_res['sentiment'] = dt_res['text'].apply(predict_sentiment)

In [ ]:
dt_res.head()

,achievements,achievements_modified,company_name,demands,id,text,sentiment
0,NaN,NaN,NaN,Уборка территории,445ad275-7a83-11ec-9ed5-57fc951f3846,. Уборка территории.,35
1,NaN,NaN,NaN,Выполняет работы по очистке территорий от сухо...,444f5e15-f88e-11ed-9b14-05dc90903fb8,. Выполняет работы по очистке территорий от су...,35
2,NaN,NaN,NaN,- проведение своевременной уборки групповой ко...,4448c865-1afe-11ee-9f4d-9586bb63c653,. - проведение своевременной уборки групповой ...,4
3,NaN,NaN,NaN,Погрузка товара,444601b5-0ea7-11ee-9332-9586bb63c653,. Погрузка товара.,8
4,NaN,NaN,NaN,Погрузка разгрузка,4445e898-140c-11ef-a0f7-d549be31d974,. Погрузка разгрузка.,8


In [ ]:
import json

# Load the label mapping
with open('/content/drive/MyDrive/Competitions/label_mapping.json', 'r') as f:
  label_mapping = json.load(f)

# Function to map sentiment to job title
def map_sentiment_to_job_title(sentiment):
  return label_mapping.get(str(sentiment), "Unknown")

dt_res['predicted_job_title'] = dt_res['sentiment'].apply(map_sentiment_to_job_title)
dt_res.head()

,achievements,achievements_modified,company_name,demands,id,text,sentiment,predicted_job_title
0,NaN,NaN,NaN,Уборка территории,445ad275-7a83-11ec-9ed5-57fc951f3846,. Уборка территории.,35,сторож
1,NaN,NaN,NaN,Выполняет работы по очистке территорий от сухо...,444f5e15-f88e-11ed-9b14-05dc90903fb8,. Выполняет работы по очистке территорий от су...,35,сторож
2,NaN,NaN,NaN,- проведение своевременной уборки групповой ко...,4448c865-1afe-11ee-9f4d-9586bb63c653,. - проведение своевременной уборки групповой ...,4,инженер
3,NaN,NaN,NaN,Погрузка товара,444601b5-0ea7-11ee-9332-9586bb63c653,. Погрузка товара.,8,сборщик
4,NaN,NaN,NaN,Погрузка разгрузка,4445e898-140c-11ef-a0f7-d549be31d974,. Погрузка разгрузка.,8,сборщик


# Linear regression

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# Загружаем токенизатор и модель BERT
tokenizer = BertTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = BertModel.from_pretrained("cointegrated/rubert-tiny2")
model.to(device)

# Функция для токенизации множества текстовых столбцов и получения BERT эмбеддингов
def get_bert_embeddings_batch(text_columns, batch_size=32):
    all_embeddings = []

    # Разделяем данные на батчи
    for start_idx in range(0, len(text_columns), batch_size):
        end_idx = min(start_idx + batch_size, len(text_columns))
        batch_text = text_columns.iloc[start_idx:end_idx].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

        # Токенизируем батч
        inputs = tokenizer(batch_text.tolist(), return_tensors='pt', padding=True, truncation=True, max_length=128)

        # Переносим токенизированные данные на GPU
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Получаем эмбеддинги без вычисления градиента
        with torch.no_grad():
            outputs = model(**inputs)

        # Извлекаем эмбеддинги по CLS-токену (первый токен)
        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.append(cls_embeddings)

        # Очистка ненужных переменных и сбор мусора
        del inputs, outputs
        torch.cuda.empty_cache()

    # Объединяем все батчи в один массив
    return np.vstack(all_embeddings)

In [ ]:
df_clear_nun = dt_sal.dropna(axis=1, how='all')

keep_columns = [
    'academic_degree', 'education_speciality', 'required_experience', 'regionName',
    'salary', 'hardSkills', 'softSkills', 'professionalSphereName',
    'industryBranchName', 'company_name', 'full_company_name', 'company_business_size',
    'vacancy_name', 'published_date', 'schedule_type', 'bonus_type',
    'additional_requirements', 'position_requirements', 'position_responsibilities',
    'languageKnowledge', 'additional_premium', 'transport_compensation', 'status'
]

existing_columns = [col for col in keep_columns if col in df_clear_nun.columns]

df_filt = df_clear_nun[existing_columns]

n = 5

df_filt.head(n)

,education_speciality,required_experience,regionName,hardSkills,softSkills,professionalSphereName,industryBranchName,full_company_name,company_business_size,vacancy_name,published_date,schedule_type,bonus_type,additional_requirements,position_requirements,position_responsibilities,languageKnowledge,additional_premium,transport_compensation,status
0,NaN,0.0,Город Москва,"[{""hard_skill_name"":""контроль исполнения докум...","[{""owner_id"":""f47e56d8-391d-11ef-8f5c-e7d0d2cf...","Продажи, закупки, снабжение, торговля",NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПРАК...",SMALL,офис менеджер,2024-07-03T15:07:19+0300,Полный рабочий день,NaN,NaN,<p>- Уверенный пользователь ПК</p> <p>- знание...,<p>- Работа в офисе</p> <p>- <strong>выполнени...,[],NaN,NaN,ACCEPTED
1,NaN,0.0,Смоленская область,[],[],"Транспорт, автобизнес, логистика, склад, ВЭД",NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТРАК...",SMALL,диспетчер автомобильного транспорта,2024-07-03T13:55:08+0300,Полный рабочий день,NaN,NaN,<ul> <li>Знание 3-х иностранных языков (англий...,<p>оформление и выдача транспортных документов...,[],NaN,NaN,ACCEPTED
2,NaN,0.0,Краснодарский край,[],[],"Спорт, фитнес, салоны красоты",NaN,"муниципальное бюджетное учреждение ""Спортивная...",SMALL,Конюх,2024-07-03T15:54:54+0300,Неполный рабочий день,NaN,NaN,NaN,<p>Уход за лошадьми</p>,[],NaN,NaN,ACCEPTED
3,NaN,0.0,Кемеровская область - Кузбасс,[],[],"Государственная служба, некоммерческие организ...",NaN,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ""КОЛОНИЯ-ПОСЕЛ...",MIDDLE,Заместитель главного бухгалтера,2024-07-03T13:00:55+0300,Полный рабочий день,QUARTERLY,NaN,<p>образование не ниже среднего специального</p>,<ul> <li> <p>руководить работой по подготовке ...,[],10000.0,NaN,ACCEPTED
4,NaN,3.0,Брянская область,[],[],"Строительство, ремонт, стройматериалы, недвижи...",NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТРИМ...",SMALL,Сметчик,2024-07-03T15:19:33+0300,Полный рабочий день,NaN,NaN,<p>Знание сметных программных комплексов (прио...,"<p>Подготовка и проверка строительных смет, по...",[],NaN,NaN,ACCEPTED


In [ ]:
text_columns = dt_sal.select_dtypes(include=['object', 'string']).columns.to_list()
numeral_columns = dt_sal.select_dtypes(include=['int', 'float']).columns.to_list()


features = text_columns + numeral_columns

In [ ]:
numerical_features = dt_sal[numeral_columns].values


X = np.concatenate([text_embeddings, numerical_features], axis=1)

NameError: name 'text_embeddings' is not defined

In [ ]:
text_embeddings = get_bert_embeddings_batch(df_filt[text_columns])

KeyError: "['id', 'accommodation_type', 'measure_type', 'busy_type', 'change_time', 'code_external_system', 'code_professional_sphere', 'company_code', 'contact_person', 'data_ids', 'date_create', 'date_modify', 'education', 'metro_ids', 'is_mobility_program', 'need_medcard', 'original_source_type', 'other_vacancy_benefit', 'required_certificates', 'required_drive_license', 'retraining_capability', 'retraining_grant', 'social_protected_ids', 'source_type', 'vacancy_address_additional_info', 'vacancy_address', 'vacancy_address_house', 'vacancy_benefit_ids', 'visibility', 'company'] not in index"

In [ ]:
numerical_features = dt_sal[numeral_columns].values
X = np.concatenate([text_embeddings, numerical_features], axis=1)

scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X)
imputer = SimpleImputer(strategy='mean')
X_test_scaled = imputer.fit_transform(X_test_scaled)

In [ ]:
dt_sal_cleaned = pd.read_csv('/content/drive/MyDrive/Competitions/SALARY_CLEANED.csv')

<ipython-input-28-266d4c43e41c>:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  dt_sal_cleaned = pd.read_csv('/content/drive/MyDrive/Competitions/SALARY_CLEANED.csv')


In [ ]:
dt_sal.shape

(1956, 75)

In [ ]:
dt_sal_cleaned.shape

(631117, 21)

In [ ]:
missing_columns = list(set(dt_sal_cleaned.columns) - set(dt_sal.columns))

print("Columns present in dt_sal_cleaned but not in dt_sal:")
print(missing_columns)

Columns present in dt_sal_cleaned but not in dt_sal:
['salary']


In [ ]:
path = '/content/drive/MyDrive/Competitions/SAL_linear_model.pkl'
model = joblib.load(path)

In [ ]:
# Предсказание на тестовой выборке
y_pred = model.predict(X_test_scaled)

print(f"Predictions: {y_pred}")

dt_sal['salary'] = y_pred

ValueError: X has 324 features, but LinearRegression is expecting 314 features as input.

# Submission

In [ ]:
def create_submission_part(test_df, name_of_predict_column, value):
  submission = pd.DataFrame([])
  submission['id'] = test_df['id']
  submission[name_of_predict_column] = [value] * len(submission)
  return submission

In [ ]:
def create_submission_res(test_df, name_of_predict_column, value):
    submission_job_title = pd.DataFrame([])
    submission_job_title['id'] = dt_res['id']
    submission_job_title['job_title'] = dt_res['predicted_job_title']
    submission_job_title['task_type'] = 'RES'
    return submission_job_title

def create_submission_sal(test_df, name_of_predict_column, value):
    submission_salary = pd.DataFrame([])
    submission_salary['id'] = dt_sal['id']
    submission_salary['salary'] = value['salary']
    submission_salary['task_type'] = 'SAL'
    return submission_salary


In [ ]:
dt_sal['salary'] = 20000

In [ ]:
submission_RES_part = create_submission_res(dt_res, 'job_title', dt_res)
submission_SAL_part = create_submission_sal(dt_sal, 'salary', dt_sal)

In [ ]:
submission_RES_part.head()

,id,job_title,task_type
0,445ad275-7a83-11ec-9ed5-57fc951f3846,сторож,RES
1,444f5e15-f88e-11ed-9b14-05dc90903fb8,сторож,RES
2,4448c865-1afe-11ee-9f4d-9586bb63c653,инженер,RES
3,444601b5-0ea7-11ee-9332-9586bb63c653,сборщик,RES
4,4445e898-140c-11ef-a0f7-d549be31d974,сборщик,RES


In [ ]:
submission_SAL_part.head()

,id,salary,task_type
0,f47e56d8-391d-11ef-8f5c-e7d0d2cf29b1,20000,SAL
1,f90390db-391d-11ef-974d-d549be31d974,20000,SAL
2,f8fcb308-391d-11ef-b4d9-cb26dff57dd7,20000,SAL
3,f936fce8-391d-11ef-b4d9-cb26dff57dd7,20000,SAL
4,fb1580e3-391d-11ef-ab54-ef6fdced905e,20000,SAL


In [ ]:
submission = pd.concat([submission_RES_part, submission_SAL_part])

In [ ]:
# Сохраняем submission в csv файл
submission.to_csv('submission_RES.csv', index=False)